In [3]:
from keras.datasets import boston_housing

In [2]:
(trainData, trainTargets), (testData, testTargets) = boston_housing.load_data()

In [3]:
trainData.shape, testData.shape

((404, 13), (102, 13))

In [4]:
mean = trainData.mean(axis=0)
trainData -= mean
std = trainData.std(axis=0)
trainData /= std

testData -= mean
testData /= std

In [5]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(trainData.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [6]:
import numpy as np

k=4
numValSamples = len(trainData) // k
numEpochs = 100
allScores = []
for i in range(k):
    print('처리 중인 폴드 #', i)
    valData = trainData[i * numValSamples: (i + 1) * numValSamples]
    valTargets = trainTargets[i * numValSamples: (i + 1) * numValSamples]
    
    partialTrainData = np.concatenate([trainData[:i * numValSamples],
                                       trainData[(i + 1) * numValSamples:]],
                                      axis=0)
    partialTrainTargets = np.concatenate([trainTargets[:i * numValSamples],
                                          trainTargets[(i + 1) * numValSamples:]],
                                         axis=0)
    model = build_model()
    model.fit(partialTrainData, partialTrainTargets,
              epochs=numEpochs, batch_size=1, verbose=0)
    valMse, valMae = model.evaluate(valData, valTargets, verbose=0)
    allScores.append(valMae)

처리 중인 폴드 # 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
처리 중인 폴드 # 1
처리 중인 폴드 # 2
처리 중인 폴드 # 3


In [7]:
allScores

[2.0936729518493804, 2.1549262504766484, 2.9525714817613657, 2.397717522512568]

In [ ]:
numEpochs = 500
allMaeHistories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    valData = trainData[i * numValSamples: (i + 1) * numValSamples]
    valTargets = trainTargets[i * numValSamples: (i + 1) * numValSamples]
    partialTrainData = np.concatenate([trainData[:i * numValSamples],
                                       trainData[(i + 1) * numValSamples:]],
                                      axis=0)
    partialTrainTargets = np.concatenate([trainTargets[:i * numValSamples],
                                          trainTargets[(i + 1) * numValSamples:]],
                                         axis=0)
    
    model = build_model()
    history = model.fit(x=partialTrainData, y=partialTrainTargets,
                        validation_data=(valData, valTargets),
                        epochs=numEpochs, batch_size=1, verbose=0)
    maeHistory = history.history['valMeanAbsoluteError']
    allMaeHistories.append(maeHistory)

처리중인 폴드 # 0


In [1]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0']
